In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
from datetime import date, datetime
import matplotlib.pyplot as plt
from string import Template
import os
import re
from collections import defaultdict
import random
import json
import asyncio
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline

print (np.__version__)
from tensortrade.feed import Stream
import tensortrade as tt

1.19.2


In [2]:
from datetime import datetime, timedelta

dateString = "2013/12/21"
dateFormatter = "%Y/%m/%d"
datetime.strptime(dateString, dateFormatter)
test_symbols = ['2330', '3008', '2317', '2412']
DEBUG = False

loop = asyncio.get_event_loop()

In [3]:
# async pipline Test brokerId fetch  
async def fetch_brokerId():
    URL = 'https://fubon-ebrokerdj.fbs.com.tw/z/js/zbrokerjs.djjs'
    
    resp = requests.get(URL)
    resp.encoding = 'cp950' # 中文
    return resp.text
  
async def post_brokerId():
    global BkrMap
    text =  await fetch_brokerId()
    reBkr = re.compile(r".*g_BrokerList = \'(.*)\'", re.MULTILINE|re.DOTALL)
    reBkr2Id = re.compile(r"(.*),(.*)", re.DOTALL)
    BkrGrps = reBkr.match(text).group(1).split(';') # slice to broker groups
    BkrMap = defaultdict(lambda: defaultdict(str))

    for bkrgrp in BkrGrps:
        bkrlist = bkrgrp.split('!') # slice to sub broker
        for i, bkr in enumerate(bkrlist):
            bmt = reBkr2Id.match(bkr)
            if bmt:
                top_bkr = bmt.group(2) if i == 0 else top_bkr
                BkrMap[top_bkr].update({bmt.group(2): bmt.group(1)}) #group(1) = b

    # debug            
    #for k in sorted(BkrMap.keys())[:1]:
    #    print(BkrMap[k])
loop = asyncio.get_event_loop()
loop.run_until_complete(post_brokerId())
#post_brokerId()

In [4]:
# random pick BHID, SBHIDS
def random_pick_BHIDs():
    global BkrMap
    test_BHIDS = list(BkrMap[random.choice(list(BkrMap.keys()))].items())
    print (test_BHIDS)
    BHID = test_BHIDS[0][1]
    SBHID = random.choice(test_BHIDS[1:])[1] if len(test_BHIDS) > 2 else BHID # if have sub SBHIDS
    #print (BHID, SBHID)
    return (BHID, SBHID)
    

In [5]:
BHID = '5380' # 台新
SBHID = '0035003300380041' # 台新建北

if not DEBUG:
    BHID, SBHID = random_pick_BHIDs()
    
print (BHID, SBHID)

[('法銀巴黎', '8900')]
8900 8900


In [6]:
#http://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=2330&BHID=9800&b=9800
# get broker stock position detail put/call volume
def get_broker_sub_symbol_position(BHID, SBHID, symbol):
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=$SYMBOL&BHID=$BHID&b=$SBHID')
    URLPar = {
        'SYMBOL': symbol,
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 't01'})[0]
    values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
    ddates = list(map(lambda x: datetime.strptime(str(x.text), dateFormatter), table.find_all('td', {'class': 't4n0'})))
    #print (values)
    #print (len(values))
    calls = [values[i] for i in range(0, len(values), 4)]  #買
    puts = [values[i+1] for i in range(0, len(values), 4)] #賣
    BkrPutCall = {
        'date': ddates,
        'symbol': symbol,
        'puts': puts,
        'calls': calls
    }
    return  pd.DataFrame(BkrPutCall)

In [7]:
symbol = '2330'
if not DEBUG:
    symbol = random.choice(test_symbols)

BkrPutCallpf = get_broker_sub_symbol_position(BHID, SBHID, symbol)
BkrPutCallpf[:4]

,date,symbol,puts,calls
0,2021-05-11,2330,202,288
1,2021-05-10,2330,2,159
2,2021-05-07,2330,0,1
3,2021-05-06,2330,255,121


In [8]:
#https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=1440&b=1440&c=E&d=1
def get_broker_all_symbol_position(BHID, SBHID):
    
    # parser GenLink2stk('AS3481','群創');
    reSymbol2Id = re.compile(r".*GenLink2stk\(\'AS(.*)\',\'(.*)\'\);", re.M|re.DOTALL)  
    
    #parser <a href="javascript:Link2Stk('00677U');">00677U富邦VIX</a>
    # no support yet
    #reSymbol2Id2 = re.compile(r".*Link2Stk\(\'(.*)\'\);\"\>(.*)\<\/a\>", re.M|re.DOTALL)
    
    def get_symbol(text): 
        smt = reSymbol2Id.match(text)
        if smt:
            return smt.group(1)
            
    def get_puts_calls(table, Bkrlist):
        values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
        symbols = list(map(lambda x:  get_symbol(str(x.text)), table.find_all('td', {'class': 't4t1'})))
        
        for i, symbol in enumerate(symbols):
            if symbol == None:
                continue
            Bkrlist.append({
            'symbol': symbol,
            'calls': values[i*3+0], #買
            'puts': values[i*3+1]   #賣
            })
        
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=$BHID&b=$SBHID&c=E&d=1')
    URLPar = {
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    tables = soup.find_all('table', {'class': 't0'})
    Bkrlist = list()
    get_puts_calls(tables[0], Bkrlist) # 買超
    get_puts_calls(tables[1], Bkrlist) # 賣超
    
    return pd.DataFrame(Bkrlist)

In [9]:
BHID2Symbolpf = get_broker_all_symbol_position(BHID, SBHID)
BHID2Symbolpf.head()

,symbol,calls,puts
0,2834,607,16
1,3481,594,93
2,2887,272,0
3,1102,165,5
4,1301,140,0


In [10]:
import asyncio
import motor.motor_asyncio
from datetime import date, datetime, timedelta
import pprint
import sys

sys.path.insert(0, os.path.abspath('../'))
import abupy
from abupy.UtilBu.ABuDateUtil import str_to_datetime

NumExpr defaulting to 8 threads.


In [11]:
client = motor.motor_asyncio.AsyncIOMotorClient('mongodb://localhost:27017')
loop = asyncio.get_event_loop()

In [12]:
# spider cmd
#%scrapy crawl StockHisTraderSpider --logfile test.log
yy, mm, dd = date.today().isoformat().split("-")[0:3]
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
#cdate = '2021-03-09'
dd = str_to_datetime(cdate)
symbol = '3362'

In [13]:
db = client['symbol-db']
collection = db['StockHisTrader']

async def do_find_StockHisTrader_one():
    document = await db['StockHisTrader'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    

loop.run_until_complete(do_find_StockHisTrader_one())

{'_id': ObjectId('609a7cd8a4792174629d505f'),
 'date': datetime.datetime(2021, 5, 11, 0, 0),
 'symbol': '3362',
 'traders': [{'call': 411.0, 'put': 0.0, 'tdname': '兆豐-中壢', 'weight': 39.41},
             {'call': 4.0, 'put': 147.0, 'tdname': '凱基-台北', 'weight': 13.71},
             {'call': 50.0, 'put': 0.0, 'tdname': '玉山-台中', 'weight': 4.79},
             {'call': 0.0, 'put': 105.0, 'tdname': '元大-華山', 'weight': 10.07},
             {'call': 31.0, 'put': 0.0, 'tdname': '摩根大通', 'weight': 2.97},
             {'call': 7.0, 'put': 83.0, 'tdname': '凱基-市政', 'weight': 7.29},
             {'call': 30.0, 'put': 0.0, 'tdname': '玉山-高雄', 'weight': 2.88},
             {'call': 0.0, 'put': 33.0, 'tdname': '陽信-石牌', 'weight': 3.16},
             {'call': 20.0, 'put': 0.0, 'tdname': '統一-新竹', 'weight': 1.92},
             {'call': 6.0, 'put': 34.0, 'tdname': '新光', 'weight': 2.68},
             {'call': 19.0, 'put': 0.0, 'tdname': '統一', 'weight': 1.82},
             {'call': 8.0, 'put': 30.0, 'tdname': '大展

In [14]:
async def do_find_StockHisData_one():
    document = await db['StockHisData'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    
loop.run_until_complete(do_find_StockHisData_one())

{'C': 69.2,
 'H': 72.0,
 'L': 69.2,
 'O': 72.0,
 'V': 1047.0,
 '_id': ObjectId('609a77a718f01ac2e6220a22'),
 'date': datetime.datetime(2021, 5, 11, 0, 0),
 'symbol': '3362'}


In [15]:
async def do_find_StockMargion_one():
    document = await db['StockHisMargin'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    
loop.run_until_complete(do_find_StockMargion_one())

{'MagPurchBalance': 4402.0,
 'MagPurchBuy': 38.0,
 'MagPurchCashRepay': 1.0,
 'MagPurchLimit': 27886.0,
 'MagPurchRatio': 15.79,
 'MagPurchSell': 176.0,
 'ShortDivMagRatio': 2.61,
 'ShortSaleBalance': 115.0,
 'ShortSaleBuy': 20.0,
 'ShortSaleCashRepay': 3.0,
 'ShortSaleSell': 0.0,
 '_id': ObjectId('609a86aa437c8c047adb4107'),
 'date': datetime.datetime(2021, 5, 11, 0, 0),
 'symbol': '3362'}


In [30]:
yy, mm, dd = date.today().isoformat().split("-")[0:3]
#dd = 19
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
day_offset = 5
today = str_to_datetime(cdate) 
test_date = today - timedelta(days=day_offset)

C_MIN, C_MAX = 50, 1000 #close Min:Max
V_MAX = 500 # volume >= V_max
broker_sampleD = 30 # broker sample [n]Days
candidates = []

In [34]:

async def do_find_rank_calls_broker_symbol():
    # test find max count of call symbols at sample time window
    pipeline = [
        {"$match": { "date": { "$lte": end_date } } },
        {"$match": { "date": { "$gte": start_date } } },
        {"$unwind": "$traders" },
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts": { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },  
                     "max_weight": { "$max": "$traders.weight" } 
        }},
        
        {"$project": {"diff_calls_puts": {"$round" : [{"$subtract": ["$sum_puts", "$sum_calls"]}, 1]}, 
                "sum_puts": "$sum_puts", "sum_calls": "$sum_calls" , "max_weight": "$max_weight", "date": "$date"} },
        
        {"$match":{ "$expr": {"$and": [
                    { "$gte": ["$diff_calls_puts", 500 ] },
                    { "$gte": ["$max_weight", 40 ] }]}
        }},
        
        {"$sort": { "max_weight": -1, "diff_calls_puts": -1, "sum_calls": -1, "sum_puts": 1 } },
        {"$limit": 50 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline, allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        cursor.update({
            'symbol': cursor['_id']['symbol'],
            'tdname': cursor['_id']['tdname']
        })
        cursor.pop('_id', None)
        results.append(cursor)
    df = pd.DataFrame(results)
    return df

rdf = []
for i in range(day_offset):
    start_date = test_date + timedelta(days=i)
    end_date = start_date + timedelta(days=1)
    df = loop.run_until_complete(do_find_rank_calls_broker_symbol())     
    rdf.append(df)

rdf = pd.concat(rdf)
rdf.sort_values(by='max_weight', ascending=False)

,diff_calls_puts,sum_puts,sum_calls,max_weight,symbol,tdname
0,763.0,763.0,0.0,90.08,1410,永豐金-板橋
0,2827.0,3659.0,832.0,86.63,00638R,元大證券
0,1236.0,1428.0,192.0,85.66,00645,永豐金證券
1,1236.0,1428.0,192.0,85.66,00645,永豐金證券
1,6815.0,6815.0,0.0,64.56,3051,凱基-台北
...,...,...,...,...,...,...
18,789.0,1617.0,828.0,41.66,00675L,元大證券
15,1751.0,2016.0,265.0,41.43,00877,凱基-台北
10,1659.0,1875.0,216.0,41.43,00877,凱基-台北
16,516.0,516.0,0.0,40.76,1592,凱基-高雄


In [18]:
# filter rank[:10], 全市場,買家對個股成交比重 
async def do_find_max_calls_weight_symbol():
    # test find max count of call symbols at sample time window
    pipeline = [
        {"$match": { "date": { "$lte": datetime.now()} } },
        {"$match": { "date": { "$gte": datetime.now() - timedelta(days=20) } } },
        {"$unwind": "$traders" },
        
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts":  { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },
                    "sum_weights": { "$sum": "$traders.weight" },
                    "max_weight": { "$max": "$traders.weight" }
                   }  },
        
        {"$project": {"avg_weights": {"$round" : [{"$divide": ["$sum_weights", 20]}, 1]}, 
                      "diff_calls_puts": {"$round" : [{"$subtract": ["$sum_calls", "$sum_puts"]}, 1]}, 
                      "sum_puts": "$sum_puts", "sum_calls": "$sum_calls", "max_weight": "$max_weight" } },
        
         {"$match":{ "$expr": {"$and": [
                        { "$gte": ["$diff_calls_puts", 100 ] },
                        { "$gte": ["$avg_weights", 10 ] }]
                    }}},
             
        {"$sort": { "diff_calls_puts": -1, "max_weight": -1, "sum_calls": -1, "sum_puts": 1 } },
        {"$limit": 20 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline, allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        cursor.update({
            'symbol': cursor['_id']['symbol'],
            'tdname': cursor['_id']['tdname']
        })
        cursor.pop('_id', None)
        results.append(cursor)
    df = pd.DataFrame(results)
    return df
df = loop.run_until_complete(do_find_max_calls_weight_symbol())     


In [19]:
# filter rank[:10], 全市場,賣家對個股成交比重 
async def do_find_max_puts_weight_symbol():
    # test find max count of put symbols at sample time window
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_puts/total_volume = total_weight
        {"$match": { "date": { "$lte": datetime.now()} } },
        {"$match": { "date": { "$gte": datetime.now() - timedelta(days=20) } } },
        {"$unwind": "$traders" },
        
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts":  { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },
                    "sum_weights": { "$sum": "$traders.weight" },
                    "max_weight":  { "$max": "$traders.weight" }
                   }  },
        
        {"$project": {"avg_weights": {"$round" : [{"$divide": ["$sum_weights", 10]}, 1]}, 
                      "diff_puts_calls": {"$round" : [{"$subtract": ["$sum_puts", "$sum_calls"]}, 1]}, 
                      "sum_puts": "$sum_puts", "sum_calls": "$sum_calls" } },
        
         {"$match":{ "$expr": {"$and": [
                        { "$gte": ["$diff_puts_calls", 100 ] },
                        { "$gte": ["$avg_weights", 10 ] }]
                    }}},
             
        {"$sort": { "avg_weights": -1, "diff_calls_puts": -1, "sum_puts": -1, "sum_calls": 1 } },
        {"$limit": 5 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline, allowDiskUse=True)
    async for cursor in collect: # clean cursor ptr and store to list
        pprint.pprint(cursor)
        
loop.run_until_complete(do_find_max_puts_weight_symbol())  

{'_id': {'symbol': '00638R', 'tdname': '元大證券'},
 'avg_weights': 55.1,
 'diff_puts_calls': 4380.0,
 'sum_calls': 5641.0,
 'sum_puts': 10021.0}
{'_id': {'symbol': '8482', 'tdname': '凱基-三重'},
 'avg_weights': 52.6,
 'diff_puts_calls': 359.0,
 'sum_calls': 384.0,
 'sum_puts': 743.0}
{'_id': {'symbol': '00762', 'tdname': '永豐金證券'},
 'avg_weights': 48.5,
 'diff_puts_calls': 1573.0,
 'sum_calls': 313.0,
 'sum_puts': 1886.0}
{'_id': {'symbol': '00763U', 'tdname': '凱基'},
 'avg_weights': 43.7,
 'diff_puts_calls': 2171.0,
 'sum_calls': 4521.0,
 'sum_puts': 6692.0}
{'_id': {'symbol': '00710B', 'tdname': '凱基-台北'},
 'avg_weights': 40.6,
 'diff_puts_calls': 2674.0,
 'sum_calls': 477.0,
 'sum_puts': 3151.0}


In [20]:
yy, mm, dd = date.today().isoformat().split("-")[0:3]
#dd = 19
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
today = str_to_datetime(cdate)

C_MIN, C_MAX = 50, 1000 #close Min:Max
V_MAX = 500 # volume >= V_max
broker_sampleD = 30 # broker sample [n]Days
candidates = []

In [35]:
# filter,rank[:N],全市場, D[t] bar 收長紅, 買家對個股成交比重 
async def find_bar_higher_max_calls_symbol():
    # test find max count of call symbols at sample time window
    pipeline = [
       # find jump spot 
      {"$lookup": {
           "from": "StockHisData",
           "pipeline": [
              {"$match":{ "$expr": {"$and": [
                        { "$lte": ["$C", C_MAX ] },
                        { "$gte": ["$C", C_MIN ] },
                        { "$ne": ["$O", "$H"]},
                        { "$ne": ["$O", "$L"]},
                        { "$gte": ["$V", V_MAX]},
                        { "$gte": ["$C", "$O"]},
                        { "$gte": [{"$divide": ["$H", "$L"]}, 1.075]}, # 
                        { "$lte" : ["$date", datetime.now() ]},
                        { "$gte" : ["$date", datetime.now() - timedelta(days=2) ]}
                       ]}}},
              {"$project": {"_id": 0 } } # no _id to next pipeline
           ],
           "as": "p0_item"
      }},
      {"$unwind":"$p0_item"}, 
      {"$match":{"$expr":{"$eq":["$p0_item.symbol", "$symbol"]}}},
      {"$match":{"$expr":{"$and":[
          { "$lte": ["$date", datetime.now() ] },
          { "$gte": ["$date", datetime.now() - timedelta(days=broker_sampleD) ] }
      ]}}},
      {"$unwind": "$traders" },
      {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts": { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },
                    "max_call
                    "tD_C": { "$first": "$p0_item.C"},
                    "tD_O": { "$first": "$p0_item.O"},
                    "tD_H": { "$first": "$p0_item.H"},
                    "tD_L": { "$first": "$p0_item.L"},
                    "tD_V": { "$first": "$p0_item.V"},
                    "tD":   { "$first": "$p0_item.date"},
                    "broker_sampleD": { "$first": broker_sampleD }
                 }},
      {"$sort": { "tD": -1, "sum_calls": -1, "sum_puts": 1, "tD_C": -1, "tD_V": -1 } },  
      #{"$limit": 1 }     
      ]
    collect = db['StockHisTrader'].aggregate(pipeline,allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        symbol_colls[cursor['_id']['symbol']].append(cursor['_id']['tdname'])
        candidates.append(cursor)
        
loop.run_until_complete(find_bar_higher_max_calls_symbol())

SyntaxError: EOL while scanning string literal (<ipython-input-35-87c9e74d37ae>, line 34)

In [ ]:
# filer,rank[:N],全市場, D[t] bar 跳空漲, 買家對個股成交比重 
async def do_find_jump_max_call_symbols():
    # test find max count of call symbols at sample time window
    pipeline = [
      # find jump spot 
      {"$lookup": {
           "from": "StockHisData",
           "pipeline": [
              {"$match":{ "$expr": {"$and": [
                        { "$lte": ["$C", C_MAX ] },
                        { "$gte": ["$C", C_MIN ] },
                        { "$ne": ["$O", "$H"]},
                        { "$ne": ["$O", "$L"]},
                        { "$gte": ["$V", V_MAX]},
                        { "$lte" : ["$date", datetime.now() ]},
                        { "$gte" : ["$date", datetime.now() - timedelta(days=2) ]}
                       ]}}},
              {"$project": {"_id": 0 } } # no _id to next pipeline
           ],
           "as": "p0_item"
      }},
      {"$lookup": {
          "from": "StockHisMargin",
          "pipeline": [
              {"$match":{ "$expr": {"$and": [
                        { "$lte": ["$ShortDivMagRatio", 2 ] },
                        { "$lte" : ["$date", datetime.now() ]},
                        { "$gte" : ["$date", datetime.now() - timedelta(days=2) ]}
                       ]}}},
              {"$project": {"_id": 0 } } # no _id to next pipeline
          ],
          "as": "p1_item"
      }},
      {"$unwind":"$p0_item"},  
      {"$match":{"$expr":{"$eq":["$p0_item.symbol", "$symbol"]}}},
      {"$unwind":"$p1_item"},   
      {"$match":{"$expr":{"$eq":["$p1_item.symbol", "$symbol"]}}},  
      {"$limit": 1 }   
      ]
    collect = db['StockHisTrader'].aggregate(pipeline)
    async for cursor in collect: # clean cursor ptr and store to list
        pprint.pprint(cursor)
        #candidates.append(cursor)
        
loop.run_until_complete(do_find_jump_max_call_symbols())

In [ ]:
test_symbols = []
for it in candidates:
    if it['_id']['symbol'] not in test_symbols:
        test_symbols += [it['_id']['symbol']] 

test_symbols[:10]

In [ ]:
short_sampleD = 20
long_sampleD = 50
symbol = '3362'
yy, mm, dd = date.today().isoformat().split("-")[0:3]
#dd = 19
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
today = str_to_datetime(cdate)

short_endD = today #datetime.now()
short_startD = datetime.now() - timedelta(days=short_sampleD)    
long_endD = datetime.now()
long_startD = datetime.now() - timedelta(days=long_sampleD)  

In [ ]:
async def get_symbol_OHLCV_df(symbol, startD, endD):
    pipeline = [
        {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
         {"$project" : { "_id": 0 } }
        ]
    collect = db['StockHisData'].aggregate(pipeline)
    items = await collect.to_list(100)
    df = pd.DataFrame(items)
    df = df.set_index('date')
    return df
 
df0 = loop.run_until_complete(get_symbol_OHLCV_df(symbol, short_startD, short_endD))
df0 = df0[['O']]
df0

In [ ]:
async def get_symbol_Margion_df(symbol, startD, endD):
    pipeline = [
        {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
         {"$project" : { "_id": 0 } }
        ]
    collect = db['StockHisMargin'].aggregate(pipeline)
    items = await collect.to_list(100)
    df = pd.DataFrame(items)
    df = df.set_index('date')
    return df
 
df1 = loop.run_until_complete(get_symbol_Margion_df(symbol, short_startD, short_endD))
df1

In [ ]:
async def get_symbol_top_traders(symbol, startD, endD):
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_calls/total_volume = total_weight
        {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
        {"$unwind": "$traders" },
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts": { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" } }  },
        {"$sort": { "sum_calls": -1, "sum_puts": 1 } },
        {"$limit": 10 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline)  
    tops = []
    async for cursor in collect: # 
        tops.append(cursor['_id']['tdname'])
    return tops
 
top_calls= loop.run_until_complete(get_symbol_top_traders(symbol, short_startD, short_endD))
top_calls[:7]

In [ ]:
async def get_symbol_top_trader_df(symbol, tdname, startD, endD):
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_calls/total_volume = total_weight
        {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
        {"$unwind": "$traders" },
                {"$match":{ "$expr": {"$and": [
                { "$eq": ["$traders.tdname", tdname ] }
            ]}}},
        {"$project" : { "_id": 0, "date": "$date", "put": "$traders.put", "call": "$traders.call" } }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline)  
    items = await collect.to_list(100)
    df = pd.DataFrame(items)
    df = df.set_index('date')
    return df

tdfs = []
for i, top_call in enumerate(top_calls[:7]):
    tdf = loop.run_until_complete(get_symbol_top_trader_df(symbol, top_call, short_startD, short_endD))
    tdf = tdf.rename(columns={"put": "Top{0}_put".format(i), "call": "Top{0}_call".format(i)})
    tdfs.append(tdf)
    
df2 = pd.concat(tdfs, axis=1).fillna(0)
df2

In [ ]:

tdfs = []
for symbol in test_symbols[0:10]:
    tdf = loop.run_until_complete(get_symbol_OHLCV_df(symbol, long_startD, long_endD))
    tdf = tdf[['O']]
    tdf = tdf.rename(columns={"O": "{0}_O".format(symbol)})
    tdfs.append(tdf)
    
dfx = pd.concat(tdfs, axis=1).fillna(0)
dfx

In [ ]:
test_symbols = []
for candidate in candidates:
    symbol = candidate['_id']['symbol'] 
    tD = candidate['tD']
    if symbol in test_symbols:
        continue
    test_symbols += [symbol]
    short_df = loop.run_until_complete(get_symbol_OHLCV_df(symbol, short_startD, short_endD))
    short_df['C'] = short_df['C'].astype(float)
    short_df['diff'] = short_df['C'] - short_df['C'].shift()
    nshort_df = short_df.sort_values(by='C', ascending=False)
    for i in range(1, 3): 
        if short_df['diff'][-1*i] == 0 and short_df.index[-1] in nshort_df.index[-4:-1]:
            print (short_df.tail(5))
